(sec:regex)=

# 正規表現の基礎


本節では、正規表現の基本について学ぶ。


## 正規表現とは？


正規表現 (regular expression)とは、特定の文字パターンに合致する文字列を見つける仕組みを指す。


In [1]:
import re

In [2]:
text = "Hello, world!"
pat = re.compile("Hello")
match = pat.search(text)
if match is not None:
    print(f'Matched text is "{match.group(0):s}"')
else:
    print("Pattern not matched.")

Matched text is "Hello"


正規表現は大文字と小文字を区別するので、`re.compile("hello")`のように`pat`を定義すると、マッチしなくなる。


正規表現のパターン文字列である`pat`には指定した文字列の中に正規表現とマッチする文字列が存在するかを調べる`search`の他にも

- 文字列の先頭から見て正規表現とマッチするかを調べる `match`
- 文字列が正規表現と完全にマッチするかを調べる `fullmatch`
- マッチした文字列を別の文字列で置き換える `sub`

などのメソッドが用意されている。


**`match`の使用例**


In [3]:
text = "Hello, world!"
pat = re.compile("ello")
if pat.match(text) is not None:
    print("Pattern found!")
else:
    print("Pattern not found!")

Pattern not found!


In [4]:
text = "Hello, world!"
pat = re.compile("Hello")
if pat.match(text) is not None:
    print("Pattern found!")
else:
    print("Pattern not found!")

Pattern found!


**`fullmatch`の使用例**


In [5]:
text = "Hello, world!"
pat = re.compile("Hello, world")
if pat.fullmatch(text) is not None:
    print("Pattern matches perfectly!")
else:
    print("Pattern does not match!")

Pattern does not match!


In [6]:
text = "Hello, world!"
pat = re.compile("Hello, world!")
if pat.fullmatch(text) is not None:
    print("Pattern matches perfectly!")
else:
    print("Pattern does not match!")

Pattern matches perfectly!


**`sub`の使用例**


In [7]:
text = "Hello, world!"
pat = re.compile("world")
text2 = pat.sub("Japan", text)
print(text2)

Hello, Japan!


```{admonition} 正規表現の基本
:class: note

- 正規表現は特定の文字パターンに合致する文字列を探すのに用いる
- Pythonにおいては標準ライブラリの`re`を用いる
- `re.compile`で文字パターンを定義した後、`search`や`fullmatch`を使って文字列を解析する
```


## 正規表現と決定性有限オートマトン


## 正規表現のシンタックス -初級編-


正規表現は特定の文字列を探すだけでなく、より複雑なパターンを持った文字列を探すことができる。


### 何らかの 1 文字


どこかに 1 文字、何でも良いので文字が存在することを表したい場合には、`a.c`のように`.`を用いる。この正規表現は例えば`adc`や`aBc`など、`.`の部分に何らか 1 文字がはいるような文字列とマッチするが、`a.c`のように`.`に対応する場所に 2 文字以上ある場合にはマッチしない。


In [8]:
pat = re.compile("a.c")
print(pat)
print("adc :", pat.fullmatch("adc") is not None)
print("aBc :", pat.fullmatch("aBc") is not None)
print("adBc :", pat.fullmatch("adBc") is not None)

re.compile('a.c')
adc : True
aBc : True
adBc : False


### 文字の繰り返し


a, b, c の 3 文字だけからなる単語が存在するかどうかを調べたい場合には、文字の繰り返しを表す`*`あるいは`+`を用いる。例えば、`aaabcccc`のような文字列とマッチさせたい場合`a+bc+`のように繰り返したい文字の後に`+`あるいは`*`をつける。


In [9]:
text = "aaabcccc"
pat = re.compile("a+bc+")
print(pat)
print(text, ":", pat.fullmatch(text) is not None)

re.compile('a+bc+')
aaabcccc : True


同じ文字の繰り返しを表す`+`と`*`は、その繰り返し回数の見方に違いがある。`+`は直前の文字が**1 回以上繰り返す**場合にのみマッチするのに対し、`*`は**0 回以上の繰り返し**にもマッチする。従って、`ab+`というパターンは`a`にはマッチせず、`ab`にはマッチする一方で、`ab*`というパターンは`a`にマッチし、なおかつ`ab`にもマッチする。


In [10]:
text1 = "a"
text2 = "ab"

In [11]:
pat = re.compile("ab+")
print(pat)
print(text1, ":", pat.fullmatch(text1) is not None)
print(text2, ":", pat.fullmatch(text2) is not None)

re.compile('ab+')
a : False
ab : True


In [12]:
pat = re.compile("ab*")
print(pat)
print(text1, ":", pat.fullmatch(text1) is not None)
print(text2, ":", pat.fullmatch(text2) is not None)

re.compile('ab*')
a : True
ab : True


加えて、回数を制限した文字の繰り返しを定義することもできる。例えば、`abbccc`は検出したいが、`c`が 4 回以上繰り返す場合や`c`が含まれない場合は除外したいという場合には、繰り返し回数は 1-3 回に指定して、`abc{1,3}`のような文字パターンを定義することができる。


In [13]:
pat = re.compile("abc{1,3}")
print("abcc :", pat.fullmatch("abcc") is not None)
print("abcccc :", pat.fullmatch("abcccc") is not None)

abcc : True
abcccc : False


### 文字セット


ここまでの例では、`+`や`*`といった文字の繰り返しは、特定の 1 文字にしか適用していなかった。しかし、Python に限らず、正規表現には特定の範囲の文字を扱う**文字セット**が用意されている。

例えば、`[a-z]`のように書くと、小文字のアルファベット全てにマッチする文字パターンを表すことができる。これに`+`や`*`を組み合わせると小文字アルファベットだけで構成されている単語かどうかをチェックすることができる。


In [14]:
pat = re.compile("[a-z]+")
print("Hello : ", pat.fullmatch("Hello") is not None)
print("hello : ", pat.fullmatch("hello") is not None)

Hello :  False
hello :  True


また、Python の正規表現では ASCII コード表で連続する文字種を文字セットの始点と終点に指定することができるため、`[x-z]`のように書くと、小文字で x, y, z のいずれかにマッチするような文字パターンを作ることもできる。


文字セットを表す`[...]`の内側には複数の文字や文字の範囲を含めることができ、例えば、大文字、小文字の全てのアルファベットと、0-9 の数字を含めたい場合、`[a-zA-Z0-9]`のような文字パターンを作ることができる。これに加えて、さらに、ハイフン (`-`)やスペース (` `)を含めたい場合には `[a-zA-Z0-9- ]`のように書けばよい。


In [15]:
text = "2-1 Naka Kunitachi-shi"
pat = re.compile("[a-zA-Z0-9- ]+")
print(text, ":", pat.fullmatch(text) is not None)

2-1 Naka Kunitachi-shi : True


また、文字セットは、そのセットに含まれない全ての文字に対してマッチさせることもできる。その場合は、文字セットの先頭に`^`を追加する。例えば、数字でない全ての文字にマッチさせたい場合には `[^0-9]` のように正規表現を定義する。


In [16]:
text = "There is a pen."
pat = re.compile("[^0-9]+")
print(text, ":", pat.fullmatch(text) is not None)

There is a pen. : True


In [17]:
text = "There is 20 pens."
pat = re.compile("[^0-9]+")
print(text, ":", pat.fullmatch(text) is not None)

There is 20 pens. : False


### 最短マッチ、最長マッチ


上記のような文字の繰り返しパターンを含むような文字列を検索する場合、特に何もしなければ、マッチする文字列のうち、最長ものが検出される。例えば、`t`から始まって`e`終わるような大文字・小文字アルファベットからなる単語を検索したい場合を考えよう。

この場合、ここまでの例にならって正規表現のパターンを作成すると以下のようになるだろうか。


In [18]:
pat = re.compile("[tT].+[e]")

この時、以下のような文字列が与えられたとする。


In [19]:
text = "Take drops in a temple."

この文字列に対して、先ほどの正規表現パターンを検索してみると、次のような結果となる。


In [20]:
match = pat.search(text)
print(match[0])

Take drops in a temple


もちろん、これはこれで正しい挙動であり、実際には正規表現パターンを`"[tT][a-z]+[e]"`のようにするべきだろう。しかし、アルファベット以外にも多くの文字が含まれる場合には、文字セットの定義が複雑になるため、パターンにマッチする最小の文字列を検索できれば便利である。

このような場合には文字の繰り返しを表す`+`や`*`の後に`?`を付け加えて、以下のように正規表現パターンを定義する。


In [21]:
pat = re.compile("[tT].+?[e]")

これを先ほどの文字列`text`から検索すると、結果は次のようになる。


In [22]:
match = pat.search(text)
print(match[0])

Take


このように、`t` (もしくは`T`)から始まり`e`で終わる部分文字列のうち、最短のものが検索できていることが分かる。なお、このようなパターンに合致する部分文字列を全て取り出したい場合には`findall`を用いる。


In [23]:
matches = pat.findall(text)
print(matches)

['Take', 'temple']


### 接頭辞、接尾辞


文字列の先頭から特定の文字パターンが出現するかどうかを調べるには、正規表現の最初に`^`を追加する。


In [24]:
pat = re.compile("^[tT].+?[e]")
text = "Take drops in a temple"
match = pat.search(text)

In [25]:
match[0]

'Take'

反対に、文字列の最後に特定の文字パターンが出現するかどうかを調べるには、正規表現の最後に`$`を追加する。


In [26]:
pat = re.compile("[tT].+?[e]$")
text = "Take drops in a temple"
match = pat.search(text)

In [27]:
match

<re.Match object; span=(0, 22), match='Take drops in a temple'>

### メタ文字


プログラミング言語によって若干の違いはあるが、正規表現には事前に定義された**メタ文字**が用意されている。例えば、`\s`というメタ文字は、半角スペース (` `)、タブ (`\t`)、キャリッジリターン (`\r`)、改ページ (`\f`)、垂直タブ (`\v`)の全てにマッチする。

| メタ文字 | 等価な正規表現   | 説明                                               |
| :------- | :--------------- | :------------------------------------------------- |
| `\d`     | `[0-9]`          | 全ての数字にマッチする                             |
| `\D`     | `[^0-9]`         | 数字以外の全ての文字にマッチする                   |
| `\s`     | `[ \t\n\r\f\v]`  | 全てのスペース記号とマッチする                     |
| `\S`     | `[^ \t\n\r\f\v]` | 全てのスペース記号でない文字とマッチする           |
| `\w`     | `[a-zA-Z0-9_]`   | 全ての英数字とアンダースコアとマッチする           |
| `\W`     | `[^a-zA-Z0-9_]`  | 英数字とアンダースコア以外の全ての文字とマッチする |


## 正規表現のシンタックス -中級編-


### サブグループ


上記の例では、マッチした文字列を取り出すために `match.group(0)`のように`group`メソッドに`0`を指定した。この`group`はマッチした文字列内にあるサブグループのことで、`0`はマッチした文章全体を示す。また`match.group(0)`は`match[0]`とも書くことができる。


In [28]:
text = '<a href="http://www.google.com">Google</a>'
pat = re.compile('<a href="(.+?)">(.+?)</a>')
match = pat.search(text)
print(match[1], match[2])

http://www.google.com Google


### 名前付きサブグループ


In [29]:
text = '<a href="http://www.google.com">Google</a>'
pat = re.compile('<a href="(?P<url>.+?)">(?P<text>.+?)</a>')
match = pat.search(text)
print(match["url"], match["text"])

http://www.google.com Google


### 日本語の取り扱い


日本語のひらがなやカタカナは、Unicode という数字コードによって整理されており、例えば「あ」の Unicode は 16 進数で`0x3042`である。これを調べるためには、文字を 10 進数の Unicode に変換する関数である`ord`と 10 進数の数字を 16 進数を表す文字列へと変換する関数である`hex`を組み合わせて、次のようにすれば良い。


In [30]:
uni = hex(ord("あ"))
print(f"Unicode of 'あ' is {uni:s}")

Unicode of 'あ' is 0x3042


反対に 16 進数の Unicode から対応する文字を調べたい場合には、`chr`関数を用いる。


In [31]:
chr(0x3042)

'あ'

Unicode 上のひらがなは小文字の「ぁ」(`0x3041`)から「ゔ」(`0x3094`)までに定義されていて、一覧にすると以下のようになる。


In [32]:
for i, c in enumerate(range(0x3041, 0x3094 + 1)):
    print(chr(c), end="")
    if (i + 1) % 10 == 0:
        print()

ぁあぃいぅうぇえぉお
かがきぎくぐけげこご
さざしじすずせぜそぞ
ただちぢっつづてでと
どなにぬねのはばぱひ
びぴふぶぷへべぺほぼ
ぽまみむめもゃやゅゆ
ょよらりるれろゎわゐ
ゑをんゔ

同じようにカタカナは「ァ」(`0x30A1`)から「ヴ」(`0x30F4`)までに定義されており、


In [33]:
for i, c in enumerate(range(0x30A1, 0x30F4 + 1)):
    print(chr(c), end="")
    if (i + 1) % 10 == 0:
        print()

ァアィイゥウェエォオ
カガキギクグケゲコゴ
サザシジスズセゼソゾ
タダチヂッツヅテデト
ドナニヌネノハバパヒ
ビピフブプヘベペホボ
ポマミムメモャヤュユ
ョヨラリルレロヮワヰ
ヱヲンヴ

In [37]:
text = "Hello日本"
pat = re.compile(".{6}")
match = pat.search(text)
print(match)

<re.Match object; span=(0, 6), match='Hello日'>


## 演習


正規表現を用いて、HTML タグから、タグ名、id、class の情報を取り出すプログラムを作成せよ。その際、id と class の指定順序は順不同であるほか、それ以外の属性値 (`align="center"`など)が指定されている場合もあることに注意せよ。

入力: `<div id="container" class="main">`  
出力: tag=div, id=container, class=main


日本語と英語の混じった文章が与えられたとき、半角英数字は 1 文字、全角の仮名文字や漢字は 2 文字として文書の長さを計算するプログラムを作成せよ。
